In [7]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [12]:
crazing_dir = "C:/Users/Checkout/Desktop/GWAR/Project/Dataset/NEU-DET/images_augmented/crazing"
inclusion_dir = "C:/Users/Checkout/Desktop/GWAR/Project/Dataset/NEU-DET/images_augmented/inclusion"
patch_dir = "C:/Users/Checkout/Desktop/GWAR/Project/Dataset/NEU-DET/images_augmented/patches"
pitted_dir = "C:/Users/Checkout/Desktop/GWAR/Project/Dataset/NEU-DET/images_augmented/pitted_surface"
rolled_dir = "C:/Users/Checkout/Desktop/GWAR/Project/Dataset/NEU-DET/images_augmented/rolled-in_scale"
scratch_dir = "C:/Users/Checkout/Desktop/GWAR/Project/Dataset/NEU-DET/images_augmented/scratches"

# Training all 60 images per defect

In [9]:
def load_dataset(directory):
    data = []
    labels = []
    for img_name in os.listdir(directory):
        if 'flippedHorz' in img_name:
            img_path = os.path.join(directory, img_name)
            img = load_img(img_path, target_size=(150, 150))
            img_array = img_to_array(img) / 255.0  
            data.append(img_array)
            labels.append(0 if directory.endswith('crazing') else
                          1 if directory.endswith('inclusion') else
                          2 if directory.endswith('patches') else
                          3 if directory.endswith('pitted_surface') else
                          4 if directory.endswith('rolled-in_scale') else
                          5) 
    return np.array(data), np.array(labels)

# Training 8 images per defect

In [24]:
import random

def load_eight_dataset(directory, num_samples=8):
    data = []
    labels = []
    for img_name in os.listdir(directory):
        if 'flippedHorz' in img_name:
            img_path = os.path.join(directory, img_name)
            if os.path.isfile(img_path):  # Check if it's a file
                img = load_img(img_path, target_size=(150, 150))
                img_array = img_to_array(img) / 255.0
                data.append(img_array)
                labels.append(0 if directory.endswith('crazing') else
                          1 if directory.endswith('inclusion') else
                          2 if directory.endswith('patches') else
                          3 if directory.endswith('pitted_surface') else
                          4 if directory.endswith('rolled-in_scale') else
                          5)
    
    
    
    # Randomly select num_samples images from the data
    if len(data) > num_samples:
        selected_indices = random.sample(range(len(data)), num_samples)
        data = [data[i] for i in selected_indices]
        labels = [labels[i] for i in selected_indices]
    
    return np.array(data), np.array(labels)


In [13]:
crazing_data, crazing_labels = load_dataset(crazing_dir)
inclusion_data, inclusion_labels = load_dataset(inclusion_dir)
patch_data, patch_labels = load_dataset(patch_dir)
pitted_data, pitted_labels = load_dataset(pitted_dir)
rolled_data, rolled_labels = load_dataset(rolled_dir)
scratch_data, scratch_labels = load_dataset(scratch_dir)

In [26]:
data = np.concatenate((crazing_data, inclusion_data, patch_data, pitted_data, rolled_data, scratch_data), axis=0)
labels = np.concatenate((crazing_labels, inclusion_labels, patch_labels, pitted_labels, rolled_labels, scratch_labels), axis=0)

In [27]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

In [28]:
data_resized = []
for img in data:
    img_resized = tf.image.resize(img, (224, 224))
    data_resized.append(img_resized)

# Convert the resized images to a numpy array
data_resized = np.array(data_resized)

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dense(6, activation='softmax')  
])

model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])


model.fit(data_resized, labels, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.0702 - loss: 1.9177 - val_accuracy: 0.5000 - val_loss: 2.8286
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 340ms/step - accuracy: 0.8602 - loss: 0.8184 - val_accuracy: 0.6000 - val_loss: 2.0062
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 340ms/step - accuracy: 0.8882 - loss: 0.3925 - val_accuracy: 0.6000 - val_loss: 0.9400
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 343ms/step - accuracy: 1.0000 - loss: 0.0653 - val_accuracy: 0.9000 - val_loss: 0.3186
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 340ms/step - accuracy: 1.0000 - loss: 0.0427 - val_accuracy: 0.9000 - val_loss: 0.2245
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 364ms/step - accuracy: 1.0000 - loss: 0.0653 - val_accuracy: 0.9000 - val_loss: 0.2165
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 371ms/step - accuracy: 1.0000 - loss: 0.0185 - val_accuracy: 1.0000 - val_loss: 0.1739
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 351ms/step - accuracy: 1.0000 - loss: 0.0048 - val_accuracy: 1.0000 - val_loss: 0.

In [29]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

crazing_val_dir = "C:/Users/Checkout/Desktop/GWAR/Project/Dataset/NEU-DET/validation/images/crazing"
inclusion_val_dir = "C:/Users/Checkout/Desktop/GWAR/Project/Dataset/NEU-DET/validation/images/inclusion"
patch_val_dir = "C:/Users/Checkout/Desktop/GWAR/Project/Dataset/NEU-DET/validation/images/patches"
pitted_val_dir = "C:/Users/Checkout/Desktop/GWAR/Project/Dataset/NEU-DET/validation/images/pitted_surface"
rolled_val_dir = "C:/Users/Checkout/Desktop/GWAR/Project/Dataset/NEU-DET/validation/images/rolled-in_scale"
scratch_val_dir = "C:/Users/Checkout/Desktop/GWAR/Project/Dataset/NEU-DET/validation/images/scratches"

In [30]:
def predict_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0  # Normalize pixel values
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    prediction = model.predict(img_array)
   
    class_labels = ['Crazing', 'Inclusion', 'Patch', 'Pitted', 'Rolled_In', 'Scratch']
    
    # Get the predicted class index
    predicted_class_index = np.argmax(prediction)
    
    # Get the predicted class label
    predicted_class = class_labels[predicted_class_index]
    
    return predicted_class


In [31]:
def evaluate_model(model, val_crazing_dir, val_inclusion_dir, val_patch_dir, val_pitted_dir, val_rolled_dir, val_scratch_dir):
    true_labels = []
    predicted_labels = []

    # Evaluate on crazing images
    for img_name in os.listdir(val_crazing_dir):
        img_path = os.path.join(val_crazing_dir, img_name)
        true_labels.append(0)  # True label for crazing
        prediction = predict_image(img_path)
        predicted_labels.append(0 if prediction == "Crazing" else 1 if prediction == "Inclusion" else 2 if prediction == "Patch" else 3 if prediction == "Pitted" else 4 if prediction == "Rolled_In" else 5)

    # Evaluate on inclusion images
    for img_name in os.listdir(val_inclusion_dir):
        img_path = os.path.join(val_inclusion_dir, img_name)
        true_labels.append(1)  # True label for inclusion
        prediction = predict_image(img_path)
        predicted_labels.append(0 if prediction == "Crazing" else 1 if prediction == "Inclusion" else 2 if prediction == "Patch" else 3 if prediction == "Pitted" else 4 if prediction == "Rolled_In" else 5)

    for img_name in os.listdir(val_patch_dir):
        img_path = os.path.join(val_patch_dir, img_name)
        true_labels.append(2)  # True label for inclusion
        prediction = predict_image(img_path)
        predicted_labels.append(0 if prediction == "Crazing" else 1 if prediction == "Inclusion" else 2 if prediction == "Patch" else 3 if prediction == "Pitted" else 4 if prediction == "Rolled_In" else 5)
        
    for img_name in os.listdir(val_pitted_dir):
        img_path = os.path.join(val_pitted_dir, img_name)
        true_labels.append(3)  # True label for inclusion
        prediction = predict_image(img_path)
        predicted_labels.append(0 if prediction == "Crazing" else 1 if prediction == "Inclusion" else 2 if prediction == "Patch" else 3 if prediction == "Pitted" else 4 if prediction == "Rolled_In" else 5)
        
    for img_name in os.listdir(val_rolled_dir):
        img_path = os.path.join(val_rolled_dir, img_name)
        true_labels.append(4)  # True label for inclusion
        prediction = predict_image(img_path)
        predicted_labels.append(0 if prediction == "Crazing" else 1 if prediction == "Inclusion" else 2 if prediction == "Patch" else 3 if prediction == "Pitted" else 4 if prediction == "Rolled_In" else 5)
        
    for img_name in os.listdir(val_scratch_dir):
        img_path = os.path.join(val_scratch_dir, img_name)
        true_labels.append(5)  # True label for inclusion
        prediction = predict_image(img_path)
        predicted_labels.append(0 if prediction == "Crazing" else 1 if prediction == "Inclusion" else 2 if prediction == "Patch" else 3 if prediction == "Pitted" else 4 if prediction == "Rolled_In" else 5)

    # Calculate metrics
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels, average='weighted')
    recall = recall_score(true_labels, predicted_labels, average='weighted')
    confusion_mat = confusion_matrix(true_labels, predicted_labels)

    return accuracy, precision, recall, confusion_mat



In [33]:
accuracy, precision, recall, confusion_mat = evaluate_model(model, crazing_val_dir, inclusion_val_dir, patch_val_dir, pitted_val_dir, rolled_val_dir, scratch_val_dir)

print("Accuracy for training of model on 8 horizontally flipped images per defect:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print("Confusion Matrix:")
print(confusion_mat)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Accuracy for training of model on 8 horizontally flipped images per defect:
Accuracy: 0.7888888888888889
Precision: 0.8435143288084466
Recall: 0.7888888888888889
Confusion Matrix:
[[11  0  0 12 37  0]


# Checking accuracy for training 8 images per defect

In [25]:
crazing_data, crazing_labels = load_eight_dataset(crazing_dir)
inclusion_data, inclusion_labels = load_eight_dataset(inclusion_dir)
patch_data, patch_labels = load_eight_dataset(patch_dir)
pitted_data, pitted_labels = load_eight_dataset(pitted_dir)
rolled_data, rolled_labels = load_eight_dataset(rolled_dir)
scratch_data, scratch_labels = load_eight_dataset(scratch_dir)